## References

- https://github.com/huggingface/transformers/blob/5f2a3d721c514cb160c74d2f2df6b729c2f99b2d/examples/text-classification/run_tf_text_classification.py

- [Jigsaw TPU: DistilBERT with Huggingface and Keras](https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras)

- https://huggingface.co/transformers/master/training.html

In [1]:
import os
import sys

In [2]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/kaggle/kaggle-quora-question-pairs/notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/kaggle/kaggle-quora-question-pairs/notebook


In [3]:
!pip install transformers -q
!pip install datasets -q

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import transformers
import datasets
from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification,
)

In [5]:
os.makedirs("../output", exist_ok=True)

In [6]:
class Config:
    def __init__(self):
        self.debug = False
        self.output_directory = "../output"
        self.train_filepath = "../input/quora-question-pairs/train.csv"
        self.test_filepath = "../input/quora-question-pairs/test.csv"
        self.model_name = "bert-base-uncased"
        self.max_seq_length = 128
        self.batch_size = 32
        self.epochs = 3
        self.num_splits = 3
        self.name = "_".join([
            self.model_name,
            "seqlen-%d" % self.max_seq_length,
            "debug" if self.debug else "prod"
        ])
        self.weights_filepath_list = [f"{self.output_directory}/{self.name}-{kfold_index}.h5" for kfold_index in range(self.num_splits)]
        
config = Config()

In [7]:
train_df = pd.read_csv(config.train_filepath)

In [8]:
def preprocess(df):
    df.fillna("", inplace=True)
preprocess(train_df)

In [9]:
if config.debug:
    train_df = train_df.sample(300)

In [10]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [11]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name, use_fast=True)

In [12]:
# cf. https://www.kaggle.com/nandanapoorv/entity-recognition-with-tf-keras-and-huggingface
# cf. https://www.kaggle.com/cdeotte/tensorflow-roberta-0-705
def build_model():
    x_input_ids = tf.keras.Input(shape=(config.max_seq_length, ), dtype=tf.int32)
    x_attention_mask = tf.keras.Input(shape=(config.max_seq_length, ), dtype=tf.int32)
    x_token_type_ids = tf.keras.Input(shape=(config.max_seq_length, ), dtype=tf.int32)
    bert_model = transformers.TFBertModel.from_pretrained(config.model_name)
    outputs = bert_model(
        input_ids=x_input_ids,
        attention_mask=x_attention_mask,
        token_type_ids=x_token_type_ids
    )
    prediction = tf.keras.layers.Dense(1, activation="sigmoid")(outputs.pooler_output)
    model = tf.keras.Model(
        inputs=[x_input_ids, x_attention_mask, x_token_type_ids],
        outputs=[prediction]
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
        loss="binary_crossentropy"
    )
    return model

In [13]:
def to_dataset(df):
    input_shape = (len(df), config.max_seq_length)
    input_ids = np.zeros(input_shape, dtype=np.int32)
    attention_mask = np.zeros(input_shape, dtype=np.int32)
    token_type_ids = np.zeros(input_shape, dtype=np.int32)
    tokens = df.apply(lambda example: tokenizer(
        example["question1"],
        example["question2"],
        truncation=True,
        max_length=config.max_seq_length,
        padding="max_length"
    ), axis=1)
    for i, token in enumerate(tokens):
        input_ids[i] = token["input_ids"]
        attention_mask[i] = token["attention_mask"]
        token_type_ids[i] = token["token_type_ids"]
    x = (input_ids, attention_mask, token_type_ids)
    y = df["is_duplicate"].values
    return x, y

In [14]:
kf = KFold(n_splits=3, shuffle=True, random_state=777)
oof = np.zeros(len(train_df))
for kfold_index, (train_index, valid_index) in enumerate(kf.split(train_df)):
    weights_filepath = config.weights_filepath_list[kfold_index]
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            weights_filepath,
            monitor="val_loss",
            verbose=1,
            save_best_only=True,
            save_weights_only=True
        )
    ]
    print("Convert to train dataset.")
    x_train, y_train = to_dataset(train_df.iloc[train_index])
    x_valid, y_valid = to_dataset(train_df.iloc[valid_index])
    print("Buiild model")
    model = build_model()
    if kfold_index == 0:
        print(model.summary())
    print("Start train")
    model.fit(
        x_train, y_train,
        validation_data=(x_valid, y_valid),
        callbacks=callbacks,
        epochs=config.epochs,
        batch_size=config.batch_size
    )
    print("Start predict")
    model.load_weights(weights_filepath)
    oof[valid_index] = model.predict(x_valid).flatten()

Convert to train dataset.
Buiild model


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Start train
Epoch 1/3
8423/8423 [==============================] - 4531s 536ms/step - loss: 0.3452 - val_loss: 0.2583

Epoch 00001: val_loss improved from inf to 0.25829, saving model to ../output/bert-base-uncased_seqlen-128_prod-1.h5
Epoch 2/3
8423/8423 [==============================] - 4521s 537ms/step - loss: 0.1946 - val_loss: 0.2426

Epoch 00002: val_loss improved from 0.25829 to 0.24263, saving model to ../output/bert-base-uncased_seqlen-128_prod-1.h5
Epoch 3/3
8423/8423 [==============================] - 4535s 538ms/step - loss: 0.1248 - val_loss: 0.2781

Epoch 00003: val_loss did not improve from 0.24263
Start predict
Convert to train dataset.
Buiild model


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Start train
Epoch 1/3
8423/8423 [==============================] - 4554s 539ms/step - loss: 0.3467 - val_loss: 0.2450

Epoch 00001: val_loss improved from inf to 0.24502, saving model to ../output/bert-base-uncased_seqlen-128_prod-2.h5
Epoch 2/3
8423/8423 [==============================] - 4539s 539ms/step - loss: 0.1952 - val_loss: 0.2429

Epoch 00002: val_loss improved from 0.24502 to 0.24287, saving model to ../output/bert-base-uncased_seqlen-128_prod-2.h5
Epoch 3/3
8423/8423 [==============================] - 4538s 539ms/step - loss: 0.1260 - val_loss: 0.2799

Epoch 00003: val_loss did not improve from 0.24287
Start predict


In [16]:
y = train_df["is_duplicate"].values
print("positive ratio: %.4f" % np.mean(y))
print("acc: %.4f" % accuracy_score(y, oof > 0.5))

positive ratio: 0.3692
acc: 0.9006
